## Mathematical Animations WITH EASE | Ep. 05: Interactivity

This notebook contains the examples discussed in [this video](https://youtu.be/KeXBLPC1tns). The cells below contain the final state of the corresponding scene from the video. Feel free to modify the code below and play around, your changes are only local and just visible to you. To add some more new cells, you can use the "+" icon above.

**IMPORTANT NOTE:** all cells containing the `interactive_embed` method **will not run properly** in the online environment. Copy the code to a local file and use the command line interface of your local installation, `manim --renderer=opengl ...`, to experiment with interactivity.

To run cells, click on them so that they are in focus, then either the "Run" button above, or hit `<Shift + Enter>`.

In [ ]:
from manim import *
from manim.opengl import *

These are some useful basic settings for Jupyter notebooks. You can use them by executing the cell, or just skip them.

In [ ]:
config.media_width = "80%"
config.verbosity = "WARNING"

#### Basic demo of the OpenGL renderer

In [ ]:
%%manim -qm --renderer=opengl --write_to_movie OpenGLIntro

class OpenGLIntro(Scene):
    def construct(self):
        hello_world = Tex("Hello World!").scale(3)
        self.play(Write(hello_world))
        self.play(
            self.camera.animate.set_euler_angles(
                theta=-10*DEGREES,
                phi=50*DEGREES
            )
        )
        self.play(FadeOut(hello_world))
        surface = OpenGLSurface(
            lambda u, v: (u, v, u*np.sin(v) + v*np.cos(u)),
            u_range=(-3, 3),
            v_range=(-3, 3)
        )
        surface_mesh = OpenGLSurfaceMesh(surface)
        self.play(Create(surface_mesh))
        self.play(FadeTransform(surface_mesh, surface))
        self.wait()
        light = self.camera.light_source
        self.play(light.animate.shift([0, 0, -20]))
        self.play(light.animate.shift([0, 0, 10]))
        self.play(self.camera.animate.set_euler_angles(theta=60*DEGREES))
        
        # self.interactive_embed()

        # self.play(self.camera.animate.set_euler_angles(theta=0*DEGREES))
        # self.play(FadeOut(surface, shift=np.array([0, 0, -2])))

        # red_sphere = Sphere(color=RED)
        # self.play(Create(red_sphere))
        # self.play(red_sphere.animate.scale(3))

        # sphere_mesh = OpenGLSurfaceMesh(red_sphere)
        # play(Transform(red_sphere, sphere_mesh))  # graphics glitch :-)

        # self.play(self.camera.animate.set_euler_angles(phi=0, theta=0))
        

### Examples for keyboard interactivity

In [ ]:
%%manim -qm --renderer=opengl --write_to_movie InteractiveRadius

class InteractiveRadius(Scene):
    def construct(self):
        plane = NumberPlane()
        cursor_dot = Dot().move_to(3*RIGHT + 2*UP)
        red_circle = Circle(
            radius=np.linalg.norm(cursor_dot.get_center()),
            color=RED
        )
        red_circle.add_updater(
            lambda mob: mob.become(
                Circle(
                    radius=np.linalg.norm(cursor_dot.get_center()),
                    color=RED
                )
            )
        )
        self.play(Create(plane), Create(red_circle), FadeIn(cursor_dot))
        self.cursor_dot = cursor_dot
        self.interactive_embed()  # not supported in online environment

    def on_key_press(self, symbol, modifiers):
        from pyglet.window import key as pyglet_key
        if symbol == pyglet_key.G:
            self.play(
                self.cursor_dot.animate.move_to(self.mouse_point.get_center())
            )
        super().on_key_press(symbol, modifiers)

In [ ]:
%%manim -qm --renderer=opengl --write_to_movie NewtonIteration

class NewtonIteration(Scene):
    def construct(self):
        self.axes = Axes()
        self.f = lambda x: (x+6) * (x+3) * x * (x-3) * (x-6) / 300
        curve = self.axes.plot(self.f, color=RED)
        self.cursor_dot = Dot(color=YELLOW)
        self.play(Create(self.axes), Create(curve), FadeIn(self.cursor_dot))
        self.interactive_embed()  # not supported in online environment

    def on_key_press(self, symbol, modifiers):
        from pyglet.window import key as pyglet_key
        from scipy.misc import derivative
        if symbol == pyglet_key.P:
            x, y = self.axes.point_to_coords(self.mouse_point.get_center())
            self.play(
                self.cursor_dot.animate.move_to(self.axes.c2p(x, self.f(x)))
            )

        if symbol == pyglet_key.I:
            x, y = self.axes.point_to_coords(self.cursor_dot.get_center())
            # Newton iteration: x_new = x - f(x) / f'(x)
            x_new = x - self.f(x) / derivative(self.f, x, dx=0.01)
            curve_point = self.cursor_dot.get_center()
            axes_point = self.axes.c2p(x_new, 0)
            tangent = Line(
                curve_point + (curve_point - axes_point)*0.25,
                axes_point + (axes_point - curve_point)*0.25,
                color=YELLOW,
                stroke_width=2,
            )
            self.play(Create(tangent))
            self.play(self.cursor_dot.animate.move_to(self.axes.c2p(x_new, 0)))
            self.play(
                self.cursor_dot.animate.move_to(self.axes.c2p(x_new, self.f(x_new))),
                FadeOut(tangent)
            )
        
        super().on_key_press(symbol, modifiers)